# Performing the Calculation

Now that we have performed the transformation on the boavizta files, we can perform the calculation with an accurate database.<br><br>
The other data that goes into the calculation you will need to fill in here, such as:
- Number of devices refurbished by type
- Yearly carbon emissions for your facility
- Pickup location and sales location

In [ ]:
import pandas as pd

## 1 Carbon benefit per product

We'll import the boavizta CSV that we made:

In [ ]:
df_boavizta = pd.read_csv('CSVs/boavizta-product-averages.csv')

Now, assign below the number of each subcategory you wish to calculate for:

In [ ]:
All_in_One_24_greater = 0
All_in_One_24_less = 0
Average_All_in_One = 0
Average_Desktop = 0
Average_Laptop = 0
Average_Monitor = 0
Desktop_SFF = 0
Desktop_Tower = 0
Desktop_USDT = 0
Laptop_14_greater = 0
Laptop_14_lesser = 0
Monitor_33_greater = 0
Monitor_33_lesser = 0
Printer = 0
Smartphone = 0
Tablet = 0
Workstation = 0

Now, we need to assign all of the above in the dataframe:

In [ ]:
df_benefits = df_boavizta.copy()
df_benefits.set_index('subcategory',inplace=True)
df_benefits.loc['All-in-One 24+', 'counts']=All_in_One_24_greater
df_benefits.loc['All-in-One <24', 'counts']=All_in_One_24_less
df_benefits.loc['Average All-in-One', 'counts']=Average_All_in_One
df_benefits.loc['Average Desktop', 'counts']=Average_Desktop
df_benefits.loc['Average Laptop', 'counts']=Average_Laptop
df_benefits.loc['Average Monitor', 'counts']=Average_Monitor
df_benefits.loc['Desktop SFF','counts']=Desktop_SFF
df_benefits.loc['Desktop Tower','counts']=Desktop_Tower
df_benefits.loc['Desktop USDT','counts']=Desktop_USDT
df_benefits.loc['Laptop 14+','counts']=Laptop_14_greater
df_benefits.loc['Laptop <14','counts']=Laptop_14_lesser
df_benefits.loc['Monitor 33+','counts']=Monitor_33_greater
df_benefits.loc['Monitor <33','counts']=Monitor_33_lesser
df_benefits.loc['Printer','counts']=Printer
df_benefits.loc['Smartphone','counts']=Smartphone
df_benefits.loc['Tablet','counts']=Tablet
df_benefits.loc['Workstation','counts']=Workstation

And we'll get the total weight for each subcategory by timesing average weight by the value counts.<br>
This will be useful for (1) when we work out the transport emisssions, and (2) when we return the total e-waste avoided.

In [ ]:
df_benefits['total_weight'] = df_benefits['weight']*df_benefits['counts']

And we can get the benefit per subcategory:

In [ ]:
df_benefits['benefit_per_subcat'] = df_benefits['counts']*df_benefits['gwp_man_trans_eol_sum']

Now, we can get the benefit for all subcategories:

In [ ]:
total_benefit = sum(df_benefits['benefit_per_subcat'])

## 2 Refurbishing Cost

Assign your organisation's scope 1 and 2 emissions in kg CO2e to the below variables:

In [ ]:
scope_1 = 
scope_2 = 
scope_1_and_2 = scope_1 + scope_2

Assign the total number of assets per year:

In [ ]:
total_assets =

Once we have this, we can use the following function which gets the total refurb cost:

In [ ]:
refurb_per_asset = scope_1_and_2/total_assets

Then, you can find the refurb cost by taking the count of the outputs in the calculation<br>
(Here I'm assuming that this is a calculation for a customer rather than yourself. Otherwise, you could use the total assets figure.)

In [ ]:
assets_processed = df_benefits['counts'].sum()
assets_processed

In [ ]:
total_refurb = assets_processed*refurb_per_asset

## 3 Transport Cost

We'll use the google maps api to calculate the distances.<br>You'll need to enter your Google Maps API key below.<br>You can make a key here: https://developers.google.com/maps and they are free up to $200 worth of calls. 

In [ ]:
#import googlemaps #UNCOMMENT THIS LINE TO USE
#gmaps = googlemaps.Client(key='UNCOMMENT AND ENTER YOUR API KEY HERE')

You'll need to assign the following locations.<br>
Keep in mind, because the pickup and deliveries are a list you can put multiple and they will be calculated:

In [ ]:
pickup_locations = ['Full addresses that the asset came from to get to the facility']
facility_location = 'Facility full address, where refurbishing happens.'
delivery_locations = ['Full delivery address of items']

To find the total distance travelled by land, we can use this function (you'll need to uncomment the columns above and put in your API key):

In [ ]:
def to_facility_distances(pickup_locations, facility_location):
    total_distances = 0
    for location in pickup_locations:
        distance_result = gmaps.distance_matrix(pickup_locations, facility_location, mode="driving",)
        total_distances+=distance_result
    return total_distances

to_facility_distances = to_facility_distances(pickup_locations=pickup_locations,
                                                facility_location=facility_location)

def from_facility_distances(facility_location, delivery_locations):
    total_distances=0
    for location in delivery_locations:
        distance_result = gmaps.distance_matrix(facility_location, delivery_locations, mode="driving",)
        total_distances+=distance_result
    return total_distances

from_facility_distances = from_facility_distances(facility_location=facility_location,
                                                delivery_locations=delivery_locations)

To know the total emissions, we need the total weights transported.<br>
For this, we just need to sum the total weights column:

In [ ]:
total_weight = sum(df_benefits['total_weight'])

From another excel with the UK government transport emissions calculation we can determine the carbon emissions for per tonne km:

In [ ]:
df_emissions_calcs = pd.read_csv('CSVs/transport-conversions.csv').set_index('Transport Type')

Per kg kilometre is the same thing/1000.<br>
We can do this here:

In [ ]:
df_emissions_calcs['kg CO2e per kg km'] = df_emissions_calcs['kg CO2e of CO2 per tonne km']/1000

For simplicity, we'll assume an average weight for each pickup.<br>
You could do a specific weight for each pickup, but that would increase the complexity of the code a bit.<br><br>Now, we have done all the set up to get the emissions for pickups and deliveries:

In [ ]:
df_pickup_emissions=(df_emissions_calcs.loc['HGV Unspecified Type Diesel','kg CO2e per kg km']*to_facility_distances)/len(pickup_locations)

In [ ]:
df_deliveries_emissions = (df_emissions_calcs.loc['HGV Unspecified Type Diesel','kg CO2e per kg km']*to_facility_distances)/len(pickup_locations)

The emissions for transport are simply the sum of the delivery and pickup emissions:

In [ ]:
transport_emissions = df_pickup_emissions+df_deliveries_demissions

For simplicity, I've assumed that transport happens in a lorry and not by air or sea.<br>Factoring air or sea becomes more complicated to calculate the distance quickly with confidence. If you want to do that, you'll need to integrate with various other APIs.<br>

## 5 The Final Result of the Calculation

Now, we have all the components of the calculation that can be specific to your facility. We just need to add and subtract as appropriate.

In [ ]:
carbon_benefit = total_benefit + total_refurb #- transport_emissions
carbon_benefit

Assuming that you refurbish **all** of the assets, you can also get a figure for e-waste avoided in kilogrammes:

In [ ]:
total_weight

Telling a story with your data is just as important as having it.<br>To understand the data, readers usually benefit from some visualisations.<br><br>You can start with a simple bar plot by subcategory:

In [ ]:
df_benefits.loc[:,'benefit_per_subcat'].plot(kind="bar")

Just to be clear, the sum of the values in this plot will be **higher** than the final figure for carbon benefit.<br>Therefore, you may want to remove the scale figures in the y axis.

## 6 After the Guide

Now that you've gone through the steps for the guide, you know the entire methodology for the calculation and its reasoning.<br>
You can clone this repo, and use this document as a startpoint to operationalise the calculation.<br>
There's plenty more that you can add:<br>
* More specific emissions for refurbishing by asset type.
* A test which checks whether your device's specific name is in the database and uses those emissions rather than an average if it is.
* More in-depth calculations for transport, which account for travel by sea and air (the Carbon Interface API is good for this).

These are all only slightly more complicated to create  than the rest of the calculation. If you'd like to learn more, you can reach me @tomburdge3@gmail.com